In [45]:
import pandas as pd
import numpy as np


In [46]:
#init
country = 'Poland'
age_data_path = 'Data/age_data.xls'
both_sheet = 'both; 1950-2005, estimates'
female_sheet = 'f; 1950-2005, estimates'
male_sheet = 'm; 1950-2005, estimates'

unnesessary_rows = range(0,5)
# right names for columns in data sheet
columns_names = {'Reference date (as of 1 July)':'year','Total population by five-year age groups (thousands)': '0 - 4', 'Unnamed: 7': '5 - 9', 'Unnamed: 8': '10 - 14', 'Unnamed: 9': '15 - 19', 'Unnamed: 10': '20 - 24', 'Unnamed: 11': '25 - 29', 'Unnamed: 12': '30 - 34', 'Unnamed: 13': '35 - 39', 'Unnamed: 14': '40 - 44', 'Unnamed: 15': '45 - 49', 'Unnamed: 16': '50 - 54', 'Unnamed: 17': '55 - 59', 'Unnamed: 18': '60 - 64', 'Unnamed: 19': '65 - 69', 'Unnamed: 20': '70 - 74', 'Unnamed: 21': '75 - 79', 'Unnamed: 22': '80 - 84', 'Unnamed: 23': '85 - 89', 'Unnamed: 24': '90 - 94', 'Unnamed: 25': '95 - 99', 'Unnamed: 26': '100+', 'Female population by five-year age groups (thousands)': '0 - 4', 'Male population by five-year age groups (thousands)': '0 - 4'}
# names of unnesessary columns
skipped_columns = {'index', 'Index', 'Variant','Major area, region, country or area*', 'Notes', 'Country code'}



In [47]:
def extract_data(sheet):
    data = pd.read_excel(age_data_path, sheet_name=sheet, skiprows = unnesessary_rows)
    data = data[data['Major area, region, country or area*'] == country]
    data = data.rename(columns=columns_names)
    data = data.reset_index().drop(columns = skipped_columns, axis = 1)
    data.replace('-', np.nan, inplace= True)
    data = data.astype(np.float64)
    data = data.astype({'year': np.int64})
    return data

In [49]:
#get needed data
data_both = extract_data(both_sheet)
data_female = extract_data(female_sheet)
data_male = extract_data(male_sheet)

data_male

,year,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,...,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 - 94,95 - 99,100+
0,1950,1500.000,1022.000,1171.000,1204.000,1159.000,1000.000,623.000,810.000,850.000,...,392.000,312.000,218.000,140.000,90.000,65.001,NaN,NaN,NaN,NaN
1,1955,1807.000,1486.000,1014.000,1158.000,1200.000,1128.000,989.000,614.000,788.000,...,503.000,344.000,260.000,166.000,97.000,64.001,NaN,NaN,NaN,NaN
2,1960,1794.635,1784.609,1478.820,987.551,1126.911,1171.025,1123.902,942.434,608.572,...,644.665,455.176,288.746,180.466,105.272,75.188,NaN,NaN,NaN,NaN
3,1965,1453.640,1762.139,1775.118,1468.616,978.412,1109.199,1156.123,1113.193,924.499,...,756.771,590.042,389.368,225.634,129.789,79.870,NaN,NaN,NaN,NaN
4,1970,1305.028,1433.773,1763.269,1764.875,1437.990,963.079,1089.314,1144.749,1084.092,...,694.241,668.331,484.519,295.746,168.847,99.631,NaN,NaN,NaN,NaN
5,1975,1472.772,1294.376,1418.785,1746.154,1740.135,1414.738,945.963,1065.705,1112.798,...,529.665,629.683,567.225,382.745,202.373,121.041,NaN,NaN,NaN,NaN
6,1980,1658.658,1465.210,1289.754,1424.658,1709.364,1699.040,1395.736,923.899,1037.789,...,797.034,478.293,507.103,452.920,268.139,161.162,NaN,NaN,NaN,NaN
7,1985,1754.872,1648.792,1458.178,1282.027,1410.345,1687.799,1673.796,1371.321,900.055,...,919.292,710.885,406.067,419.335,297.021,192.028,NaN,NaN,NaN,NaN
8,1990,1512.917,1731.082,1651.766,1456.392,1263.358,1359.536,1652.299,1628.489,1330.256,...,930.334,815.145,601.448,317.543,283.287,233.861,NaN,NaN,NaN,NaN
9,1995,1273.475,1504.848,1724.815,1642.230,1440.725,1249.572,1342.532,1623.628,1586.828,...,864.570,820.383,681.145,465.524,218.918,160.290,67.383,16.904,2.397,0.172
